# Advent of Code 2023

https://adventofcode.com/2023

In [1]:
# A few standard library imports
import itertools, math, collections, os, functools, json, time, re, bisect
# nice printing
from IPython.display import display, clear_output, HTML

In [2]:
def getData(day, year=2023):
    if not os.path.exists("data-%d" %year):
        os.mkdir("data-%d" %year)
    if os.path.exists("data-%d/day%dinput.txt" %(year, day)):
        with open("data-%d/day%dinput.txt" %(year, day), 'r') as f:
            return f.read()
    import browsercookie, urllib.request
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(browsercookie.firefox()))
    assert b'watercrossing' in opener.open("https://adventofcode.com/%d/" %year).read()
    data = opener.open("https://adventofcode.com/%d/day/%d/input" %(year, day)).read().decode('utf-8')
    with open("data-%d/day%dinput.txt" %(year, day), 'w') as f:
        f.write(data)
    return data

In [3]:
def isNum(s: str) -> bool:
    return all(47 < ord(c) < 58 for c in s)

## Day 1

In [4]:
day1data = getData(1).splitlines()

In [5]:
sum(int(next(c for c in x if isNum(c)) + next(c for c in x[::-1] if isNum(c))) for x in day1data)

54927

In [6]:
z = list(zip(["one", "two", "three", "four", "five", "six", "seven", "eight", "nine"] + [str(x) for x in range(0, 10)], 
              [str(x) for x in range(1, 10)] + [str(x) for x in range(0, 10)]))

In [7]:
sum(int(min([(x.find(c), d) for c, d in z if c in x], key=lambda a: a[0])[1] + 
        min([(x[::-1].find(c[::-1]), d) for c, d in z if c in x], key=lambda a: a[0])[1]) for x in day1data)

54581

## Day 2

In [8]:
day2data = [(int(r.split(":")[0].split(" ")[1]), [collections.Counter(dict((y.split(" ")[1], int(y.split(" ")[0])) for y in x.split(", ")))
                                                  for x in r.split(": ")[1].split("; ")]) for r in getData(2).splitlines()]

In [9]:
sum(c for c, x in day2data if all(max(y[a] for y in x) < b for a, b in (('red', 13), ('green', 14), ('blue', 15))))

2239

In [10]:
sum(math.prod(max(c[a] for c in x) for a in ('red', 'green', 'blue')) for _, x in day2data)

83435

## Day 3

In [11]:
day3data = getData(3).splitlines()
il, jl= len(day3data), len(day3data[0])

In [12]:
sum(int(num) for i in range(il) for j in range(jl) if (j==0 or not isNum(day3data[i][j-1])) and isNum(day3data[i][j])
    and (num := next(day3data[i][j:k] for k in range(j+1,jl+1) if isNum(day3data[i][j:k]) and (k == jl or not isNum(day3data[i][k]))))
    and any(not isNum(day3data[a][b]) and day3data[a][b] != '.' for a in range(max(0,i-1), min(il,i+2)) 
            for b in range(max(0, j-1), min(jl, j+1+len(num)))))

538046

In [13]:
sum(math.prod(gears) for i in range(il) for j in range(jl) if day3data[i][j] == "*"
    and len(gears := [next(int(day3data[k][lx:ly]) for lx in range(l, max(-1, l-6), -1) for ly in range(l+1, min(jl+1, l+6))
                           if isNum(day3data[k][lx:ly]) and (lx == 0 or not isNum(day3data[k][lx-1])) and (ly == jl or not isNum(day3data[k][ly]))) 
                      for k, l in [(ix, j-1) for ix in range(max(0, i-1), min(il, i+2)) if j > 0] + 
                                  [(ix, jx) for ix in range(max(0, i-1), min(il, i+2)) for jx in range(j, min(jl, j+2))
                                   if jx == 0 or not isNum(day3data[ix][jx-1])] if isNum(day3data[k][l])]) == 2)

81709807

## Day 4

In [14]:
day4data = [tuple(set(y for y in x.split(": ")[1].split(" | ")[i].split(" ") if y) for i in range(2)) for x in getData(4).splitlines()]

In [15]:
sum(2**(len(c&w)-1) if len(c&w) else 0 for w, c in day4data)

24160

In [16]:
cards = [[1, len(c&w)] for c, w in day4data]
for i, (c, w) in enumerate(cards):
    for j in range(w):
        cards[i+j+1][0] += c
sum(x[0] for x in cards)

5659035

## Day 5

In [17]:
day5data = [[[int(y) for y in l.split()] for l in x.split(":")[1].strip().splitlines()] for x in getData(5).split("\n\n")]
day5data[0] = day5data[0][0]

In [18]:
min(functools.reduce(lambda cs, m: [next(itertools.chain(d+c-s for d, s, l in m if s <= c < s+l), c) for c in cs], day5data))

324724204

In [19]:
day5data[0] = [[range(day5data[0][2*i], day5data[0][2*i]+day5data[0][2*i+1])] for i in range(len(day5data[0])//2)]

In [20]:
def day5map(css, m):
    coveredRanges = [[range(max(s, c[0]), min(s+l, c[-1]+1)) for c in cs for d, s, l in m if min(s+l, c[-1]+1) > max(s, c[0])] for cs in css]
    destRanges = [[range(max(s, c[0])-s+d, min(s+l, c[-1]+1)-s+d) for c in cs for d, s, l in m if min(s+l, c[-1]+1) > max(s, c[0])] for cs in css]
    notMapped = []
    for tc, cr in zip(css, coveredRanges):
        for d in cr:
            tc = [x for c in tc for x in [range(c[0], min(c[-1]+1, d[0])), range(max(d[-1]+1, c[0]), c[-1]+1)] if x.stop > x.start]
        notMapped.append(tc)
    return [x+y for x, y in zip(destRanges, notMapped)]

In [21]:
min(x[0] for y in functools.reduce(day5map, day5data) for x in y)

104070862

## Day 6

In [22]:
day6data = list(zip(*[[int(x.strip()) for x in r.split(":")[1].split(" ") if x] for r in getData(6).splitlines()]))

In [23]:
math.prod(sum(1 for i in range(t) if i*t-i*i > r) for t, r in day6data)

771628

In [24]:
day6dataPart2 = [int(r.split(":")[1].strip().replace(" ","")) for r in getData(6).splitlines()]

In [25]:
class Day6:
    def __getitem__(self, key):
        return key*day6dataPart2[0] - key*key
    def __len__(self):
        return day6dataPart2[0]//2

In [26]:
(day6dataPart2[0]//2 - bisect.bisect(Day6(), day6dataPart2[1]))*2 + 1

27363861

## Day 7

In [27]:
day7data = [x.split(" ") for x in getData(7).splitlines()]

In [28]:
def day7replace(s):
    s = s.replace("T", "a").replace("J", "b").replace("Q", "c").replace("K", "d").replace("A", "e")
    c = collections.Counter(s).values()
    if 5 in c:
        s = "6" + s
    elif 4 in c:
        s = "5" + s
    elif 3 in c and 2 in c:
        s = "4" + s
    elif 3 in c:
        s = "3" + s
    elif collections.Counter(c)[2] == 2:
        s = "2" + s
    elif 2 in c:
        s = "1" + s
    return int(s, base=16)

In [29]:
day7sorted = sorted(day7data, key=lambda x: day7replace(x[0]))

In [30]:
sum((i+1)*int(bid) for i, (h, bid) in enumerate(day7sorted))

249726565

In [31]:
def day7replacePart2(s):
    s = s.replace("T", "a").replace("J", "1").replace("Q", "c").replace("K", "d").replace("A", "e")
    js = collections.Counter(s)['1'] 
    c = collections.Counter(s.replace("1", "")).values()
    if js == 5 or 5 in c or max(c)+js == 5:
        s = "6" + s
    elif 4 in c or max(c) + js == 4:
        s = "5" + s
    elif 3 in c and 2 in c or (collections.Counter(c)[2] == 2 and js == 1):
        s = "4" + s
    elif 3 in c or (2 in c and js == 1) or js == 2:
        s = "3" + s
    elif collections.Counter(c)[2] == 2:
        s = "2" + s
    elif 2 in c or js == 1:
        s = "1" + s
    return int(s, base=16)

In [32]:
day7sortedPart2 = sorted(day7data, key=lambda x: day7replacePart2(x[0]))

In [33]:
sum((i+1)*int(bid) for i, (h, bid) in enumerate(day7sortedPart2))

251135960

## Day 8

In [34]:
day8data = (getData(8).split("\n\n")[0], 
            dict((x.split(" = ")[0], x.split(" = ")[1].strip("()").split(", ")) for x in getData(8).split("\n\n")[1].split("\n") if x))

In [35]:
c, cn = 0, "AAA"
while cn != "ZZZ":
    cn = day8data[1][cn][0 if day8data[0][c % len(day8data[0])] == 'L' else 1]
    c += 1
c

22411

In [36]:
cns = [x for x in day8data[1].keys() if x.endswith("A")]
orders = [[0] for _ in range(len(cns))]
for i, cn in enumerate(cns):
    c = 0
    while len(orders[i]) != 4:
        cn = day8data[1][cn][0 if day8data[0][c % len(day8data[0])] == 'L' else 1]
        c += 1
        if cn.endswith("Z"):
            orders[i].append(c)

In [37]:
[[x[i+1]-x[i] for i in range(len(x)-1)] for x in orders]

[[16271, 16271, 16271],
 [24253, 24253, 24253],
 [13201, 13201, 13201],
 [14429, 14429, 14429],
 [18113, 18113, 18113],
 [22411, 22411, 22411]]

In [38]:
# they are all the same. I didn't expect that. But that makes it easy.

In [39]:
math.lcm(*(x[1] for x in orders))

11188774513823

## Day 9

In [40]:
day9data = [[int(x) for x in y.split(" ")] for y in getData(9).splitlines()]

In [41]:
nextVals = []
for x in day9data:
    diffs = [x[-1]]
    while not all(y == 0 for y in x):
        x = [x[i+1] - x[i] for i in range(len(x)-1)]
        diffs.append(x[-1])
    nextVals.append(sum(diffs))
sum(nextVals)

1987402313

In [42]:
nextVals = []
for x in day9data:
    diffs = [x[0]]
    while not all(y == 0 for y in x):
        x = [x[i+1] - x[i] for i in range(len(x)-1)]
        diffs.append(x[0])
    nextVals.append(sum((-1)**i*x for i, x in enumerate(diffs)))
sum(nextVals)

900

## Day 10

In [43]:
day10data = getData(10).splitlines()
tileDirs = {"-": ((0, 1), (0, -1)), "|": ((1, 0), (-1, 0)), "J": ((-1, 0), (0, -1)), 
            "7": ((0, -1), (1, 0)), "L": ((-1, 0), (0, 1)), "F": ((1, 0), (0, 1))}
nbs = [(0, 1), (0, -1), (1, 0), (-1, 0)]

In [44]:
start = next((i, j) for i in range(len(day10data)) for j in range(len(day10data[0])) if day10data[i][j] == "S")
connectIntoStart = [(i, j) for i, j, shapes in [(start[0]+1, start[1], "|JL"), (start[0]-1, start[1], "|7F"), 
                                                (start[0], start[1]-1, "-FL"), (start[0], start[1]+1, "-J7")] if day10data[i][j] in shapes]

In [45]:
loop = [start, connectIntoStart[0]]
while loop[-1] != start:
    loop.append(next(nt for i, j in tileDirs[day10data[loop[-1][0]][loop[-1][1]]] if (nt := (loop[-1][0] + i, loop[-1][1] + j)) != loop[-2]))
(len(loop) - 1 ) // 2

6697

In [46]:
loopSet = set(loop)
day10dataPart2 = [[[day10data[i][j], set()] if (i, j) in loopSet else [".", set()] for j in range(len(day10data[0]))] for i in range(len(day10data))]
for d in itertools.cycle((range(len(day10data))[::x],range(len(day10data[1]))[::y])  for x, y in [(1, 1), (1, -1), (-1, 1), (-1, -1)]):
    anyChanges = False
    for i, j in ((x, y) for x in d[0] for y in d[1]):
        if day10dataPart2[i][j][0] == ".":
            if (i in (0, len(day10data) - 1) or j in (0, len(day10data[0]) - 1) or
                any(day10dataPart2[i+k][j+l][0] == "O" or (-k, -l) in day10dataPart2[i+k][j+l][1] for k, l in nbs)):
                day10dataPart2[i][j][0], c = "O", True
        elif day10dataPart2[i][j][0] in '-|J7LF':
            nn = set((k, l) for k, l in nbs if (k, l) not in tileDirs[day10dataPart2[i][j][0]] and 
                     (i+k in (-1, len(day10data)) or j+l in (-1, len(day10data[0])) or
                      day10dataPart2[i+k][j+l][0] == "O" or (-k, -l) in day10dataPart2[i+k][j+l][1] or
       any((m, n) in day10dataPart2[i][j][1] for m, n in nbs if (m, n) not in tileDirs[day10dataPart2[i][j][0]] and not (m == -k and n == -l)) or
       any((k, l) in day10dataPart2[i+m][j+n][1] for m, n in nbs if abs(k)==abs(n) and abs(l) == abs(m))))
            if nn != day10dataPart2[i][j][1]:
                nn, anyChanges = day10dataPart2[i][j][1].update(nn), True
    if not anyChanges:
        break

In [47]:
sum(1 for i in range(len(day10data)) for j in range(len(day10data[0])) if day10dataPart2[i][j][0] == '.')

423

## Day 11

In [48]:
day11data = getData(11).splitlines()
emptyRows = [yi for yi, y in enumerate(day11data) if "#" not in y]
emptyColumns = [xi for xi in range(len(day11data[0])) if not any(y[xi] == "#" for y in day11data)]
day11data2 = [(yi + sum(1 for el in emptyRows if el < yi), xi + sum(1 for el in emptyColumns if el < xi)) 
             for yi, y in enumerate(day11data) for xi, x in enumerate(y) if x == "#"]

In [49]:
sum(abs(y2-y1)+abs(x2-x1) for (y1, x1), (y2, x2) in itertools.combinations(day11data2, 2))

9769724

In [50]:
day11data2 = [(yi + sum(999999 for el in emptyRows if el < yi), xi + sum(999999 for el in emptyColumns if el < xi)) 
             for yi, y in enumerate(day11data) for xi, x in enumerate(y) if x == "#"]

In [51]:
sum(abs(y2-y1)+abs(x2-x1) for (y1, x1), (y2, x2) in itertools.combinations(day11data2, 2))

603020563700

## Day 12

In [52]:
day12data = [(x.split(" ")[0], tuple(int(y) for y in x.split(" ")[1].split(","))) for x in getData(12).splitlines()]

In [53]:
@functools.lru_cache(maxsize=1000)
def day12recurse(s, ls):
    solution = 0
    matches = re.finditer('(?=([?#]{%d})(?!#))' %ls[0], s) #overlapping, 0-width matches
    for m in matches:
        if not '#' in s[:m.start(1)]: # can't skip any "#". No idea how to do that inside the regexp though
            remaining = s[m.start(1) + ls[0] + 1:].strip(".")
            if len(ls) == 1 and "#" not in remaining:
                    solution += 1
            elif len(ls) > 1 and sum(1 for x in remaining if x in "?#") >= sum(ls[1:]):# still possible?
                    solution += day12recurse(remaining, ls[1:])
    return solution

In [54]:
sum(day12recurse(s,ls) for s, ls in day12data)

7025

In [55]:
sum(day12recurse("?".join(s for _ in range(5)), ls*5) for s, ls in day12data)

11461095383315

## Day 13

In [56]:
day13data = [x.splitlines() for x in getData(13).split("\n\n") if x]

In [57]:
res = 0
for x in day13data:
    try:
        res += next(i for i in range(1,len(x[0])) if all(y[i-j] == y[i+j-1] for y in x for j in range(1,1+min(i, len(x[0])-i))))
    except StopIteration:
        res += 100*next(i for i in range(1,len(x)) if all(a == b for j in range(1, 1+min(i, len(x)-i)) for a, b in zip(x[i-j], x[i+j-1])))
res

30802

In [58]:
res = 0
for x in day13data:
    try:
        res += next(i for i in range(1,len(x[0])) if sum(y[i-j] != y[i+j-1] for y in x for j in range(1,1+min(i, len(x[0])-i))) == 1)
    except StopIteration:
        res += 100*next(i for i in range(1,len(x)) if sum(a != b for j in range(1, 1+min(i, len(x)-i)) for a, b in zip(x[i-j], x[i+j-1])) == 1)
res

37876